
# 🧪 Autonomous Activity: Dimensionality Reduction on Embryo Development Timelapse

In this hands-on activity, you will apply a variety of dimensionality reduction techniques to analyze real microscopy data. The dataset contains time-lapse images of normal and mutant embryos. Each image stack has approximately 450 frames.

---

**🎯 Objectives:**
- Explore, visualize, and preprocess multi-frame `.tif` images.
- Test and compare different data normalization strategies (e.g., [0,1] scaling vs StandardScaler).
- Use PCA, SVD, t-SNE, UMAP, and Autoencoders to extract and visualize developmental trajectories.
- Identify biological differences in developmental dynamics between embryo types.
- Reflect on the advantages, limitations, and behaviors of each technique.

📁 **Dataset:** [Google Drive Link](https://drive.google.com/drive/folders/1_qxqm-v5yCrme3pAW2rjyOOXIeQDuV54?usp=drive_link)



## 1. Load and Explore the Dataset

Each `.tif` file contains ~450 grayscale frames. Your first task is to:
- Load the 3 `.tif` files using `tifffile.imread`.
- Normalize each image stack using two strategies:
  - [0, 1] Min-Max normalization
  - Standardization using `StandardScaler`
- Plot a few representative frames across time for each embryo.


In [ ]:

from tifffile import imread
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler

def normalize_minmax(img):
    return (img - img.min()) / (img.max() - img.min())

def normalize_standard(img):
    flat = img.reshape(img.shape[0], -1)
    scaler = StandardScaler()
    return scaler.fit_transform(flat).reshape(img.shape)

def show_frames(stack, title, step=100):
    plt.figure(figsize=(12, 2))
    for i in range(5):
        frame = stack[i * step]
        plt.subplot(1, 5, i + 1)
        plt.imshow(frame, cmap="gray")
        plt.title(f"{title} t={i*step}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# Replace this path with the local folder where your .tif files are stored
folder_path = "path_to_downloaded_tif_folder"
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".tif")]
stacks = [imread(fp) for fp in file_paths]
names = [os.path.basename(fp).split(".")[0] for fp in file_paths]

# Show examples (min-max normalized)
for name, stack in zip(names, stacks):
    show_frames(normalize_minmax(stack), title=name)



## 2. Preprocess the Data

Flatten each frame and construct a matrix `X` with shape `(n_frames, n_pixels)` for each normalization method. Then create a label vector:

- `label = 0` → control embryo
- `label = 1` → mutant 1
- `label = 2` → mutant 2

This step will prepare the input data for dimensionality reduction.


In [ ]:

X_minmax, X_standard, y = [], [], []

for i, stack in enumerate(stacks):
    norm_minmax = normalize_minmax(stack)
    norm_standard = normalize_standard(stack)
    X_minmax.append(norm_minmax.reshape(stack.shape[0], -1))
    X_standard.append(norm_standard.reshape(stack.shape[0], -1))
    y.append(np.full(stack.shape[0], i))

X_minmax = np.vstack(X_minmax)
X_standard = np.vstack(X_standard)
y = np.concatenate(y)

print("MinMax shape:", X_minmax.shape)
print("StandardScaler shape:", X_standard.shape)
print("Labels:", np.unique(y))



> 🧪 **Experiment**: Try running all dimensionality reduction methods with both versions of the input (`X_minmax` and `X_standard`) and compare how they affect embeddings and separability.



## 3. PCA (Principal Component Analysis)

Apply PCA on both normalized datasets:
- Visualize the 2D PCA embedding colored by embryo type.
- Compare the separation between classes for `X_minmax` and `X_standard`.
- Plot the **explained variance ratio** and **cumulative variance** for each.

> 🧠 Tip: Use `PCA(n_components=2)` for plotting and `PCA(n_components=50)` to analyze cumulative variance.


In [ ]:

from sklearn.decomposition import PCA

def run_pca(X, labels, title=""):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    plt.figure()
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="tab10", s=5)
    plt.colorbar(scatter, label="Embryo type")
    plt.title(f"PCA Embedding - {title}")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()

    # Scree plot
    pca_full = PCA().fit(X)
    cum_var = np.cumsum(pca_full.explained_variance_ratio_)
    plt.figure()
    plt.plot(cum_var, marker="o")
    plt.axhline(0.95, linestyle="--", color="r")
    plt.title(f"Cumulative Explained Variance - {title}")
    plt.xlabel("Number of components")
    plt.ylabel("Cumulative variance")
    plt.grid(True)
    plt.show()

# PCA comparison
run_pca(X_minmax, y, "Min-Max Normalization")
run_pca(X_standard, y, "StandardScaler")



## 4. SVD (Singular Value Decomposition)

Apply SVD to both datasets and analyze:
- The decay of singular values on a log scale.
- The cumulative energy of singular values.
- Compare how quickly each normalization captures energy.

> 🔍 Insight: SVD reveals the inherent structure of your dataset. A sharper drop often suggests stronger linear compressibility.


In [ ]:

def plot_svd(X, title=""):
    U, S, Vt = np.linalg.svd(X - X.mean(axis=0), full_matrices=False)
    energy = np.cumsum(S**2) / np.sum(S**2)

    plt.figure()
    plt.plot(np.log10(S), marker="o")
    plt.title(f"SVD - Singular Values (log10) - {title}")
    plt.xlabel("Component")
    plt.ylabel("log10(σ)")
    plt.grid(True)
    plt.show()

    plt.figure()
    plt.plot(energy, marker="o")
    plt.axhline(0.95, linestyle="--", color="r")
    plt.title(f"SVD - Cumulative Energy - {title}")
    plt.xlabel("Component")
    plt.ylabel("Cumulative energy")
    plt.grid(True)
    plt.show()

plot_svd(X_minmax, "Min-Max")
plot_svd(X_standard, "StandardScaler")



## 5. t-SNE

Use t-SNE to capture local structure and dynamics:
- Run t-SNE with different perplexities `[5, 30, 100]`.
- Plot the 2D embeddings and analyze how clusters behave.
- Compare results between both normalized inputs.

> ⏳ Note: t-SNE is computationally intensive and sensitive to hyperparameters.


In [ ]:

from sklearn.manifold import TSNE

def run_tsne(X, labels, title=""):
    for perp in [5, 30, 100]:
        tsne = TSNE(n_components=2, perplexity=perp, init="pca", random_state=42)
        X_tsne = tsne.fit_transform(X)
        plt.figure()
        scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=labels, cmap="tab10", s=5)
        plt.title(f"t-SNE ({title}) - perplexity={perp}")
        plt.colorbar(scatter, label="Embryo")
        plt.show()

run_tsne(X_minmax, y, "Min-Max")
run_tsne(X_standard, y, "StandardScaler")



## 6. UMAP

Use UMAP for global structure visualization:
- Try combinations of `n_neighbors` and `min_dist`.
- Compare embeddings across normalization methods.
- Observe both local clustering and trajectory smoothness.

> 📌 UMAP is faster and often preserves better continuity in developmental trajectories.


In [ ]:

try:
    import umap

    def run_umap(X, labels, title=""):
        settings = [(5, 0.1), (15, 0.1), (50, 0.5)]
        for n_n, min_d in settings:
            reducer = umap.UMAP(n_neighbors=n_n, min_dist=min_d, random_state=42)
            X_umap = reducer.fit_transform(X)
            plt.figure()
            scatter = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=labels, s=5, cmap="tab10")
            plt.title(f"UMAP ({title}) - n_neighbors={n_n}, min_dist={min_d}")
            plt.colorbar(scatter, label="Embryo")
            plt.show()

    run_umap(X_minmax, y, "Min-Max")
    run_umap(X_standard, y, "StandardScaler")
except ImportError:
    print("UMAP not installed. Run: pip install umap-learn")



## 7. Autoencoder

Train a neural autoencoder to learn a 2D latent space:
- Architecture: `input → 128 → 32 → 2 → 32 → 128 → output`
- Use ReLU activations and MSE loss.
- Plot the latent 2D representations colored by embryo.

> 💡 Autoencoders are flexible nonlinear methods that may capture dynamics not seen by linear projections.


In [ ]:

try:
    import tensorflow as tf
    from tensorflow.keras import layers, models

    def run_autoencoder(X, labels, title=""):
        input_dim = X.shape[1]
        encoder = models.Sequential([
            layers.Input(shape=(input_dim,)),
            layers.Dense(128, activation="relu"),
            layers.Dense(32, activation="relu"),
            layers.Dense(2)
        ])
        decoder = models.Sequential([
            layers.Input(shape=(2,)),
            layers.Dense(32, activation="relu"),
            layers.Dense(128, activation="relu"),
            layers.Dense(input_dim)
        ])

        autoencoder = models.Sequential([encoder, decoder])
        autoencoder.compile(optimizer="adam", loss="mse")
        autoencoder.fit(X, X, epochs=20, batch_size=64, verbose=0)

        X_latent = encoder.predict(X)
        plt.figure()
        scatter = plt.scatter(X_latent[:, 0], X_latent[:, 1], c=labels, s=5, cmap="tab10")
        plt.title(f"Autoencoder Latent Space - {title}")
        plt.colorbar(scatter, label="Embryo")
        plt.show()

    run_autoencoder(X_minmax, y, "Min-Max")
    run_autoencoder(X_standard, y, "StandardScaler")
except ImportError:
    print("TensorFlow not installed. Run: pip install tensorflow")



## 8. Final Reflection

Write a short report answering the following:

1. What differences in developmental dynamics did you observe?
2. Which method best captured biologically relevant features?
3. Which normalization (Min-Max or StandardScaler) led to better embeddings?
4. At which point do mutant trajectories diverge from normal?
5. How consistent were the results across methods (PCA, t-SNE, UMAP, Autoencoders)?

> 📝 Submit this as a 1-page summary or short presentation.
